# Imports
Import all the necessary libraries required for the project.

In [43]:
# **Imports**:

# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

# Load Dataset
Load the dataset that will be used for training and testing the model.

In [44]:
# **Load Dataset**

# Load the dataset into a pandas DataFrame
dataset_path = 'sample_data/CEAS_08.csv'
df = pd.read_csv(dataset_path)

# Data Preprocessing
Perform data cleaning and preprocessing steps to prepare the data for modeling.

In [45]:
# **Data Preprocessing**
# Check for missing values
missing_values = df.isnull().sum()
df = df.dropna()
missing_values = df.isnull().sum()

In [46]:
# Print
print(missing_values)

sender      0
receiver    0
date        0
subject     0
body        0
label       0
urls        0
dtype: int64


In [55]:
print(df.shape)
X = df['body']  # Use the 'body' column as features
y = df['label']  # Use the 'label' column as the target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=0)
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Initialize and train a model (e.g., Logistic Regression)
from sklearn.linear_model import LogisticRegression # Make sure to import LogisticRegression
model = LogisticRegression() # Initialize the model
model.fit(X_train_vec, y_train) # Train the model


y_pred = model.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

(38669, 7)
Accuracy: 0.9872855788294113


In [54]:
# prompt: create fucntion to fecth the mdel test a body and return true or false

import pickle

def predict_text(text, model, vectorizer):
    # Transform the input text using the fitted vectorizer
    text_vec = vectorizer.transform([text])
    # Predict the label
    prediction = model.predict(text_vec)
    return prediction[0]  # Return the predicted label

# Example usage
sample_text = "This is a test message."
predicted_label = predict_text(sample_text, model, vectorizer)
print("Predicted Label:", predicted_label)


Predicted Label: 0


## Using Fucntion




In [69]:
sample_text = "We detected that your password has been exposed in a data breach on another platform while doing a routine check for your security. You should reset your Duolingo password immediately, using a unique password not used elsewhere."
isPishing = predict_text(test_body, model, vectorizer)
if isPishing:
    print("The body is a phishing email.")
else:
    print("The body is not a phishing email.")

The body is a phishing email.


In [70]:
import numpy as np

def get_suspect_words(text, model, vectorizer, top_n=5):
    # Transform the input text using the fitted vectorizer
    text_vec = vectorizer.transform([text])

    # Get feature names (words) from the vectorizer
    feature_names = np.array(vectorizer.get_feature_names_out())

    # Get model coefficients (importance of each word)
    coef = model.coef_[0]  # Extract coefficients for the positive class

    # Get the nonzero feature indices in the transformed input
    word_indices = text_vec.nonzero()[1]

    # Get word importance scores
    word_importance = coef[word_indices]

    # Pair words with their importance scores and sort them
    word_contributions = sorted(zip(feature_names[word_indices], word_importance), key=lambda x: abs(x[1]), reverse=True)

    # Return the top N suspect words
    return word_contributions[:top_n]

# Example usage
sample_text = "We detected that your password has been exposed in a data breach on another platform while doing a routine check for your security. You should reset your Duolingo password immediately, using a unique password not used elsewhere."
suspect_words = get_suspect_words(sample_text, model, vectorizer)

print("Top suspect words:", suspect_words)


Top suspect words: [('your', np.float64(5.114041736250909)), ('on', np.float64(-3.5255309992236312)), ('you', np.float64(2.838929881507155)), ('that', np.float64(-2.3414119366788966)), ('data', np.float64(-2.260703777375827))]
